<a href="https://colab.research.google.com/github/dajinfx/hanghaeAI2/blob/main/W5_2_MLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##과제: 다양한 형태의 입력을 가지는 LLM 서비스 개발

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dajinfx/hanghaeAI2/blob/main/W5_2_MLLM.ipynb)

In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [2]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
import openai
import json
import requests
import numpy as np
import torch
import pandas as pd
from langchain.prompts import PromptTemplate

In [4]:
gpt_model="gpt-4o"
temperature = 0.7  # 각 token을 샘플링할 때 사용하는 temperature 값입니다.
max_tokens = 4096  # 생성하는 최대 token 개수 입니다.
frequency_penalty = 0.0  # 같은 단어가 반복적으로 나오는 것을 방지하기 위한 옵션입니다.

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
    print(f" {filename} is uploaded")

with open('OpenAI_APIKey.txt', 'r') as file:
    api_key = file.read()

Saving OpenAI_APIKey.txt to OpenAI_APIKey (1).txt
 OpenAI_APIKey (1).txt is uploaded


In [6]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [7]:
file_url = "https://arxiv.org/pdf/2005.11401"

In [8]:
pip install "unstructured[pdf]"

  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.


In [10]:
!apt-get update
!apt-get install -y poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,642 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,227 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,600 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,859 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/

In [12]:
!apt-get update
!apt-get install -y tesseract-ocr

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
!pip install unstructured[local-inference] pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 10.3 MB/s eta 0:00:00


In [17]:
nltk.data.path.clear()
nltk.download('punkt', download_dir='/content/nltk_data')

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 修复路径
from langchain.schema import Document

# 使用 UnstructuredURLLoader 加载 PDF 文件
loader = UnstructuredURLLoader(urls=[file_url])
docs = loader.load()



ERROR:langchain_community.document_loaders.url:Error fetching or processing https://arxiv.org/pdf/2005.11401, exception: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
**********************************************************************



In [ ]:
# 打印加载的文档
for doc in docs:
    print(doc.page_content)

In [7]:
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

#참조
'''
#url을 unstructuredURLLoader를 사용해 불러옵니다.
# 해당 툴을 사용하면, 별다른 설정없이 url의 정보를 가져옵니다.
url = "https://spartacodingclub.kr/blog/all-in-challenge_winner"
loader = UnstructuredURLLoader(urls=[url])
docs = loader.load()
'''

loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["editedContent"]},   #Html class 명으로서 문서내의 원소명에 따라 달라질수 있다
        )
    ),

    #header_template={
    #   "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    #},
    encoding="utf-8",
)
docs = loader.load()

NameError: name 'client' is not defined

In [ ]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The image shows a serene outdoor scene featuring a wooden path or boardwalk that winds through a lush, green landscape. On either side of the path, tall grasses and some bushes are visible, indicating a natural setting, possibly a wetland or meadow. The sky is partially cloudy, with a blue hue, adding to the peaceful ambiance of the scene.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
image_path = "path_to_your_image.jpg"
base64_image = encode_image(image_path)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        }
    ],
)

In [ ]:
response.choices[0]

In [ ]:
UnstructuredFileLoader
from langchain_community.document_loaders import UnstructuredURLLoader